# Portfolio Analysis Potential Loss

## Import Modules

In [18]:
import pandas as pd
import numpy as np

## Load Data

In [19]:
df = pd.read_csv('CaseStudy_Data.csv')

## Functions

In [20]:
def potential_loss_project(mvalue_loss: float,mvalue,loan_nom,senior_loan, security = 0, incl_security_proj = False) -> float: #data, 
    """ 
    return potential loss depending on market value loss on project basis
    
    :param mvalue_loss float: market value loss, value from 0 to 1
    :param mvalue float: current market value of the property being financed
    :param loan_nom float: subordinated loan coming from company
    :param senior_loan float: loan that is preferred over subordinated loans in the event of the borrower’s insolvency
    :param bool security: security amount that is deposited as security and can be claimed by the lender in the event of a default
    :param bool incl_security_port: If set to True, security amount included in analysis
    """
    mvalue_loss_abs = mvalue * mvalue_loss
    equity = mvalue - loan_nom - senior_loan
    if equity >= mvalue_loss_abs:
        #equity covers all the losses
        return 0
    elif equity + loan_nom <= mvalue_loss_abs:
        if incl_security_proj:
            #we loose the loan but get te security amount so loss = loan - security amount
            return loan_nom - security
        else:
            #total loss of loan
            return loan_nom
        #total equity loss and more
    else:
        if incl_security_proj:
            #In case loss loan loss is smaller than security, loss is 0
            return max([0,(mvalue_loss_abs - equity) - security])
        else:
            return mvalue_loss_abs - equity

        

In [21]:
def potential_loss_portfolio(df_input, incl_security_port = False):
    """
    Calculate portfolio loss for different market value loss scenarios
    
    :param DataFrame df: Raw data to perform risk analysis on
    :param bool incl_security_port: If set to True, security amount included in analysis

    """
    df_input = df_input.iloc[:,1:]
    df_input = df_input.reset_index(drop = True)
    df_input.columns = [col.lower() for col in df_input.columns]
    
    #Add potential loss columns
    add_cols = [str(round(mv_loss,2)) for mv_loss in np.arange(0, 1.05, 0.05)]
    for col_name in add_cols:
        df_input = df_input.assign(**{col_name: ''})
    no_projects = len(df_input)-1

    #For each project calculate potential loss for every .05 step
    for project in range(0,no_projects+1):
        mvalue = df_input.loc[project,'marketvalue']
        loan_nom = df_input.loc[project,'loannominal'] 
        senior_loan = df_input.loc[project,'seniorloan']
        security_amount = df_input.loc[project,'securityamount']
        if incl_security_port:
            for mv_loss in np.arange(0, 1.05, 0.05):
                df_input.loc[project,str(round(mv_loss,2))] = potential_loss_project(round(mv_loss,2),mvalue,loan_nom,senior_loan,security_amount,incl_security_proj = True)
        else:
            for mv_loss in np.arange(0, 1.05, 0.05):
                df_input.loc[project,str(round(mv_loss,2))] = potential_loss_project(round(mv_loss,2),mvalue,loan_nom,senior_loan)
    return df_input


In [22]:
def filter_ltv_x(df,x):
    """
    filter df on any x

    :param DataFrame df: Dataframe containing losses per finance project in long format
    :param int x: value to filter on ltv
    """
    df = df[df['ltv'] <= x]
    return df


In [23]:
def potential_total_loss_portfolio(df,ltv_x_value = 100):
    """
    sum up losses to get total loss per market_value loss for the total portfolio, option to filter
    
    :param DataFrame df: Dataframe containing losses per finance project in long format
    :param int ltv_x_value: Filter to apply in case ltv_x set to True

    """
    df = filter_ltv_x(df,ltv_x_value)
    df = df.loc[:, ~df.columns.isin(['projektnumber',	'ltv',	'marketvalue',	'loannominal',	'seniorloan',	'securityamount'])]
    df_total = pd.DataFrame(df.sum()).reset_index(drop=False)
    df_total = df_total.rename(columns={'index':'mv_loss',0:'potential_loss'})
    df_total['potential_loss'] =df_total['potential_loss'].astype(float).round(0)
    df_total['mv_loss'] =df_total['mv_loss'].astype(float)
    return df_total

In [24]:
df_output = potential_loss_portfolio(df,incl_security_port = False)
df_output.head()

,projektnumber,ltv,marketvalue,loannominal,seniorloan,securityamount,0.0,0.05,0.1,0.15,...,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0
0,Finanzierung 1,82,62998289.11,5934659.07,45723938,2870999,0,0,0,0,...,5934659.07,5934659.07,5934659.07,5934659.07,5934659.07,5934659.07,5934659.07,5934659.07,5934659.07,5934659.07
1,Finanzierung 2,83,22498612.13,9017395.07,9656453,4008859,0,0,0,0,...,8549472.6115,9017395.07,9017395.07,9017395.07,9017395.07,9017395.07,9017395.07,9017395.07,9017395.07,9017395.07
2,Finanzierung 3,60,22440163.79,1993615.27,11470483,120386,0,0,0,0,...,1993615.27,1993615.27,1993615.27,1993615.27,1993615.27,1993615.27,1993615.27,1993615.27,1993615.27,1993615.27
3,Finanzierung 4,79,39505557.84,17389988.69,13819402,5695326,0,0,0,0,...,13431889.662,15407167.554,17382445.446,17389988.69,17389988.69,17389988.69,17389988.69,17389988.69,17389988.69,17389988.69
4,Finanzierung 5,72,94303113.51,57178460.73,10719781,50227221,0,0,0,0,...,25461840.6505,30176996.326,34892152.0015,39607307.677,44322463.3525,49037619.028,53752774.7035,57178460.73,57178460.73,57178460.73


In [25]:
df_final = potential_total_loss_portfolio(df_output)
df_final.head()

,mv_loss,potential_loss
0,0.00,0.0
1,0.05,0.0
2,0.10,0.0
3,0.15,0.0
4,0.20,25304552.0
